In [1]:
# Packages for Web-Scraping
from requests import get
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn

# Packages for Saving File after Scraping
import numpy as np
import pandas as pd

In [2]:
# url 1st page info, response object, and sample
# url = 'https://www.politifact.com/truth-o-meter/statements/?page=1'
url = 'https://web.archive.org/web/20180705082623/https://www.politifact.com/truth-o-meter/statements/?page=1'
response = get(url)
print(response.text[:500])

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml">
    <head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app101.us.archive.org';v.server_ms=325;archive_analytics.send_pageview({});});</script><


In [3]:
# 1st page BeautifulSoup object, confirm type
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
# finding each row of data we want to scrape, confirm type, check length
statement_containers = html_soup.find_all('div', class_ = 'scoretable__item')
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
20


In [5]:
# checking data
s = statement_containers[1]
s

<div class="scoretable__item">
<div class="statement">
<div class="meter">
<a href="/web/20180705082623/https://www.politifact.com/personalities/becoming-american-initiative/statements/byruling/mostly-true/"><img alt="Mostly True" src="https://web.archive.org/web/20180705082623im_/http://static.politifact.com.s3.amazonaws.com/rulings/tom-mostlytrue.png"/></a>
<p class="quote">Roll the tape</p>
</div>
<div class="statement__body">
<div class="mugshot">
<a href="/web/20180705082623/https://www.politifact.com/personalities/becoming-american-initiative/"><img alt=" Becoming American Initiative" src="https://web.archive.org/web/20180705082623im_/http://static.politifact.com.s3.amazonaws.com/politifact/mugs/becomingamericaninitiative.png"/></a>
</div>
<div class="statement__source"><a href="/web/20180705082623/https://www.politifact.com/personalities/becoming-american-initiative/"> Becoming American Initiative</a></div>
<p class="statement__text">
<a class="link" href="/web/20180705082623/ht

In [6]:
# statement
s.find('p', class_ = 'statement__text').get_text(strip=True)

'Says Ronald Reagan said immigrants\xa0"brought with them courage and the values of family, work, and freedom. Let us pledge to each other that we can make America great again."'

In [7]:
# statement source
s.find('div', class_ = 'statement__source').get_text(strip=True)

'Becoming American Initiative'

In [8]:
# statement link
s.find('p', class_ = 'statement__text').a["href"]

'/web/20180705082623/https://www.politifact.com/truth-o-meter/statements/2018/jul/03/becoming-american-initiative/did-ronald-reagan-say-immigrants-made-america-grea/'

In [9]:
# statement veracity
s.img["alt"]

'Mostly True'

In [10]:
# Lists to store the scraped data in
statement = []
source = []
link = []
veracity = []

# Extract data from individual container
for container in statement_containers:
# statement
    sta = container.find('p', class_ = 'statement__text').get_text(strip=True)
    statement.append(sta)
# source
    sou = container.find('div', class_ = 'statement__source').get_text(strip=True)
    source.append(sou)
# link
    lin = container.find('p', class_ = 'statement__text').a["href"]
    link.append(lin)
# veracity
    ver = container.img["alt"]
    veracity.append(ver)

In [11]:
# 1st page df
test_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  20 non-null     object
 1   source     20 non-null     object
 2   link       20 non-null     object
 3   veracity   20 non-null     object
dtypes: object(4)
memory usage: 768.0+ bytes
None


,statement,source,link,veracity
0,"Sen. Kamala Harris is ""supporting the animals ...",Donald Trump,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
1,"Says Ronald Reagan said immigrants ""brought wi...",Becoming American Initiative,/web/20180705082623/https://www.politifact.com...,Mostly True
2,"Says Democratic Senators ""demand Supreme Court...",Viral image,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
3,"""Tim Kaine doesn’t want a border at all. He wa...",Corey Stewart,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
4,"""There are a lot of private charters. And in f...",Kelda Helen Roys,/web/20180705082623/https://www.politifact.com...,Half-True
5,"""George H.W. Bush has died at 94.""",Bloggers,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
6,"""The deficit ... is coming down, and it’s comi...",Larry Kudlow,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
7,"""Watch those GDP numbers. We started off at a ...",Donald Trump,/web/20180705082623/https://www.politifact.com...,False
8,"""The European Union … they send us Mercedes, t...",Donald Trump,/web/20180705082623/https://www.politifact.com...,False
9,"""Clinton campaign official arrested by FBI on ...",Bloggers,/web/20180705082623/https://www.politifact.com...,False


In [12]:
# Testing 5 pages
pages = [str(i) for i in range(1,5)]
start_time = time()
requests = 0
for _ in range(5):
# request goes here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.38440986199934657 requests/s


In [13]:
# If above is problem-free, proceed to scrape all pages
# Scraping 740 pages takes about 4-5 hours

# Lists to store all the scraped data in
statement = []
source = []
link = []
veracity = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every page in the interval
pages = [str(i) for i in range(740)]
for page in pages:

    # Make a get request
    response = get('https://web.archive.org/web/20180705082623/https://www.politifact.com/truth-o-meter/statements/?page=' + page)

    # Pause the loop in random intervals so your IP address doesn't get banned
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the containers from a single page
    statement_containers = page_html.find_all('div', class_ = 'scoretable__item')

    for container in statement_containers:
    # statement
        sta = container.find('p', class_ = 'statement__text').get_text(strip=True)
        statement.append(sta)
    # source
        sou = container.find('div', class_ = 'statement__source').get_text(strip=True)
        source.append(sou)
    # link
        lin = container.find('p', class_ = 'statement__text').a["href"]
        link.append(lin)
    # veracity
        ver = container.img["alt"]
        veracity.append(ver)

C:\Users\sergi\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Request: 740; Status code: 404


In [14]:
# make a dataframe
politifact_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(politifact_df.info())
politifact_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14209 entries, 0 to 14208
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  14209 non-null  object
 1   source     14209 non-null  object
 2   link       14209 non-null  object
 3   veracity   14209 non-null  object
dtypes: object(4)
memory usage: 444.2+ KB
None


,statement,source,link,veracity
14207,"""Tommy Thompson is the father of welfare refor...",Tommy Thompson,/web/20180119180622/http://www.politifact.com/...,True
14208,"""Tommy Thompson created the first school choic...",Tommy Thompson,/web/20180119180622/http://www.politifact.com/...,True


In [15]:
# save dataframe to csv
politifact_df.to_csv('politifact.csv')

In [16]:
# Tutorial followed to create this scraper: 
# https://www.dataquest.io/blog/web-scraping-beautifulsoup/

In [17]:
politifact_df.shape

(14209, 4)

In [18]:
politifact_df.head()

,statement,source,link,veracity
0,"Sen. Kamala Harris is ""supporting the animals ...",Donald Trump,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
1,"Says Ronald Reagan said immigrants ""brought wi...",Becoming American Initiative,/web/20180705082623/https://www.politifact.com...,Mostly True
2,"Says Democratic Senators ""demand Supreme Court...",Viral image,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
3,"""Tim Kaine doesn’t want a border at all. He wa...",Corey Stewart,/web/20180705082623/https://www.politifact.com...,Pants on Fire!
4,"""There are a lot of private charters. And in f...",Kelda Helen Roys,/web/20180705082623/https://www.politifact.com...,Half-True
